# MTL Demo

### First, import dependencies

In [1]:
from __future__ import print_function

import random
from logging import info
import sys

from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense, merge
from keras.layers import Reshape, Convolution2D, Dropout

import __main__
__main__.__file__ = "./"
from models.ltlib import filelog
from models.ltlib import conlldata
from models.ltlib import viterbi

from models.ltlib.features import NormEmbeddingFeature, SennaCapsFeature
from models.ltlib.features import windowed_inputs
from models.ltlib.callbacks import token_evaluator, EpochTimer
from models.ltlib.layers import concat, inputs_and_embeddings
from models.ltlib.settings import cli_settings, log_settings
from models.ltlib.optimizers import get_optimizer
from models.ltlib.output import save_token_predictions

from models.config import Defaults
from sklearn.metrics import precision_recall_fscore_support, f1_score

sys.argv = ["single_task.py", "./data/CHEMDNER", "./vectorfile/bio_nlp_vec/PubMed-shuffle-win-30.bin"]

default_stdout = sys.stdout
default_stderr = sys.stderr
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = default_stdout
sys.stderr = default_stderr

Using TensorFlow backend.


### Define the class for implementation

In [2]:
class MTL:
    def __init__(self):
        self.config = cli_settings(['datadir', 'wordvecs'], Defaults)
        
    def convert_ground_truth(self, data, *args, **kwargs):  # <--- implemented PER class
        pass

    def read_dataset(self, file_dict, dataset_name):  # <--- implemented PER class
        data = conlldata.load_dir(self.config.datadir, self.config)
        return data
            
            
    def train(self, data):  # <--- implemented PER class
        if self.config.viterbi:
            vmapper = viterbi.TokenPredictionMapper(data.train.sentences)
        else:
            vmapper = None

        w2v = NormEmbeddingFeature.from_file(self.config.wordvecs,
                                             max_rank=self.config.max_vocab_size,
                                             vocabulary=data.vocabulary,
                                             name='words')
        features = [w2v]
        if self.config.word_features:
            features.append(SennaCapsFeature(name='caps'))

        data.tokens.add_features(features)
        data.tokens.add_inputs(windowed_inputs(self.config.window_size, features))

        # Log word vector feature stat summary
        #info('{}: {}'.format(self.config.wordvecs, w2v.summary()))

        inputs, embeddings = inputs_and_embeddings(features, self.config)

        seq = concat(embeddings)
        cshape = (self.config.window_size, sum(f.output_dim for f in features))

        seq_reshape = Reshape(cshape+(1,))(seq)


        # Convolutions
        conv_outputs = []
        for filter_size, filter_num in zip(self.config.filter_sizes, self.config.filter_nums):
            conv = Convolution2D(filter_num, filter_size, cshape[1],
                                 activation='relu')(seq_reshape)
            cout = Flatten()(conv)
            conv_outputs.append(cout)
        seq_covout = merge(conv_outputs, mode='concat', concat_axis=-1)


        for size in self.config.hidden_sizes:
            seq_2 = Dense(size, activation=self.config.hidden_activation)(seq_covout)
        seq_3 = Dropout(self.config.output_drop_prob)(seq_2)
        out = Dense(data.tokens.target_dim, activation='softmax')(seq_3)
        self.model = Model(input=inputs, output=out)


        optimizer = get_optimizer(self.config)
        self.model.compile(loss='categorical_crossentropy', optimizer=optimizer,
                      metrics=['accuracy'])

        callbacks = [
            EpochTimer(),
            #token_evaluator(data.train, mapper=vmapper, config=config),
            #token_evaluator(data.test, mapper=vmapper, config=config),
        ]

        percnt_keep = self.config.percent_keep
        amt_keep = len(data.train.tokens.inputs['words']) * percnt_keep
        print("Total: %s. Keeping: %s" % (len(data.train.tokens.inputs['words']), amt_keep))
        start = random.randrange(int(len(data.train.tokens.inputs['words']) - amt_keep + 1))
        end = int(start + amt_keep)
        x = data.train.tokens.inputs['words'][start:end]


        self.model.fit(
            #data.train.tokens.inputs,
            x,
            data.train.tokens.targets[start:end],
            callbacks=callbacks,
            batch_size=self.config.batch_size,
            nb_epoch=1,
            verbose=1
        )
      
    def predict(self, data, *args, **kwargs):  # <--- implemented PER class WITH requirement on OUTPUT format!
        name = "chemdner_output"
        save_token_predictions(name, data.test, self.model, conlldata.write)

        return "./prediction/"+name+".tsv"

    def evaluate(self, predictions, groundTruths, *args,
                 **kwargs):  # <--- common ACROSS ALL classes. Requirement that INPUT format uses output from predict()!
        f = open(predictions, "r")
        lines = f.readlines()
        f.close()

        ground = []
        predict = []

        for line in lines:
            if len(line) < 2:
                 continue
            tmp = line.split(" ")
            ground.append(tmp[1])
            predict.append(tmp[2].strip())

        labels = list(set(predict))

        eval = precision_recall_fscore_support(ground, predict, labels=labels)
        test_score = eval[2]

        for idx, label in enumerate(labels):
            print("{} Precision:{:.2f}% Recall:{:.2f}% F1_score:{:.2f}% ".format(label, eval[0][idx]*100, eval[1][idx]*100, eval[2][idx]*100))

        eval_summary = precision_recall_fscore_support(ground, predict, average='macro', labels=labels)
        
        return eval_summary


    def save_model(self, filepath):
        pass
    

    def load_model(self, filepath):
        pass


### Initial the MTL class instance

In [3]:
MTL_instance = MTL()

./:md5:4246eea35eedb1d89e578715e4764f5f/1a7b4c315b31a418d861cf5eb9b7fdc9
{
    "batch_size": 200,
    "datadir": "./data/CHEMDNER",
    "encoding": "utf-8",
    "epochs": 20,
    "evaluate_every": 5000,
    "evaluate_min": 0,
    "filter_nums": [ 100, 100, 100 ],
    "filter_sizes": [ 3, 4, 5 ],
    "fixed_wordvecs": true,
    "hidden_activation": "relu",
    "hidden_sizes": [ 1200 ],
    "iobes": false,
    "learning_rate": 0.0001,
    "max_tokens": null,
    "max_vocab_size": 1000000,
    "optimizer": "adam",
    "output_drop_prob": 0.75,
    "percent_keep": 1.0,
    "token_level_eval": false,
    "train_steps": 20000,
    "verbosity": 1,
    "viterbi": false,
    "window_size": 7,
    "word_features": false,
    "wordvecs": "./vectorfile/bio_nlp_vec/PubMed-shuffle-win-30.bin"
}


### Load data

Load data calling the read_dataset method and pass the path. 

In [4]:
read_data = MTL_instance.read_dataset("./data/CHEMDNER", None)

### Train model

After the data is loaded, pass them to the train method

As for a demo, we only train for 1 epoch

In [5]:
MTL_instance.train(read_data)

./:setting embedding_lr_multiplier not defined
./:incomplete weights, added 1 missing


Instructions for updating:
keep_dims is deprecated, use keepdims instead


./:From /home/samtom/project/MTL-env/local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:1047: calling reduce_prod (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


./:From /home/samtom/project/MTL-env/local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:2385: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
keep_dims is deprecated, use keepdims instead


./:From /home/samtom/project/MTL-env/local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:1108: calling reduce_mean (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Total: 604216. Keeping: 604216.0
Epoch 1/1
604216/604216 [==============================] - 123s - loss: 0.1960 - acc: 0.9497   

./:Ep: 1 123s (start 2019-05-01 03:22:05, end 2019-05-01 03:24:08)


### Prediction

Pass the test data which is contained in the read_data object.

Call the predict function and get the output file path

In [6]:
output_file = MTL_instance.predict(read_data)
print("Output file has been created at: {}".format(output_file))

Output file has been created at: ./prediction/chemdner_output.tsv


In [7]:
f = open(output_file, "r")
lines = f.readlines()
f.close()

Simply open and read in the output file

We can see the sample of prediction as follow

For each line, the first column is the word, the second is the groundtruth entity, and the third is the prediction entity

In [8]:
lines[30:40]

['beneficial O O\n',
 'substances O O\n',
 'e O O\n',
 'g O O\n',
 'docosahexaenoic B-FAMILY B-TRIVIAL\n',
 'acids I-FAMILY I-FAMILY\n',
 'but O O\n',
 'also O O\n',
 'harmful O O\n',
 'compounds O O\n']

### Evaluation

Call the evaluate function and pass the output_file in it.

From the output, the Precision, Recall and F1_score are printed

At the last row, the average score of the whole model is printed.

Since the demo only train for 1 epoch, the F1_score is not ideal

In [9]:
score = MTL_instance.evaluate(output_file, None)
print("Average Score:")
print("Precision:{:.2f}% Recall:{:.2f}% F1_score:{:.2f}% ".format(score[0]*100, score[1]*100, score[2]*100))

B-FORMULA Precision:76.98% Recall:64.22% F1_score:70.02% 
I-FORMULA Precision:62.49% Recall:63.39% F1_score:62.94% 
B-TRIVIAL Precision:73.38% Recall:68.94% F1_score:71.09% 
I-FAMILY Precision:61.53% Recall:37.62% F1_score:46.69% 
B-SYSTEMATIC Precision:77.15% Recall:56.79% F1_score:65.43% 
O Precision:97.63% Recall:99.58% F1_score:98.59% 
I-TRIVIAL Precision:70.82% Recall:24.13% F1_score:35.99% 
B-FAMILY Precision:61.75% Recall:42.21% F1_score:50.15% 
B-ABBREVIATION Precision:69.45% Recall:35.50% F1_score:46.98% 
I-SYSTEMATIC Precision:71.53% Recall:75.75% F1_score:73.58% 
Average Score:
Precision:72.27% Recall:56.81% F1_score:62.15% 
